<a href="https://colab.research.google.com/github/Blackman9t/Advanced-Data-Science/blob/master/AssignmentML1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the first assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Just execute all cells one after the other and you are done - just note that in the last one you have to update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

In [6]:
!wget https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet
#https://raw.githubusercontent.com/IBM/coursera/master/coursera_ml/AssignmentML1.ipynb

--2020-02-17 20:04:30--  https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/coursera_ml/a2.parquet [following]
--2020-02-17 20:04:30--  https://raw.githubusercontent.com/IBM/coursera/master/coursera_ml/a2.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59032 (58K) [application/octet-stream]
Saving to: ‘a2.parquet’

a2.parquet          100%[===================>]  57.65K  --.-KB/s    in 0.02s   

2020-02-17 20:04:30 (2.37 MB/s) - ‘a2.parquet’ saved [59032/59032]



Now let's load pyspark and its dependencies in Colab

In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

Next we set up a Spark Context if none exists

In [8]:
from pyspark import SparkConf, SparkContext
try:
    conf = SparkConf().setMaster('local').setAppName('my_app')
    sc = SparkContext(conf = conf)
    print('SparkContext initialised successfully!')
except Exception as e:
    print(type(e),e)

<class 'ValueError'> Cannot run multiple SparkContexts at once; existing SparkContext(app=my_app, master=local) created by __init__ at <ipython-input-3-57a84026bec8>:4 


Next we create a SparkSession named spark

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('my_app').getOrCreate()
spark

Let's now ontinue processing the parquet file

In [10]:
df=spark.read.load('a2.parquet')

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+-----+-----------+-------------------+-------------------+-------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|
+-----+-----------+-------------------+-------------------+-------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|
|    0|25769803830| 178.62396382387422| -47.07529438881511|  84.38310769821979|
|    0|25769803831|  85.03128805189493|-4.3024316644854546|-1.1841857567516714|
|    0|34359738411| 26.786262674736566| -46.33193951911338| 20.880756008396055|
|    0| 8589934592|-16.203752396859194| 51.080957032176954| -96.80526656416971|
|    0|25769803852|   47.2048142440404| 

In [11]:
spark.catalog.listTables()

[Table(name='df', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [12]:
type(df)

pyspark.sql.dataframe.DataFrame

In [13]:
df.printSchema()

root
 |-- CLASS: integer (nullable = true)
 |-- SENSORID: long (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- Z: double (nullable = true)



In [0]:
!rm -Rf a2_m1.json

In [0]:
df = df.repartition(1)
df.write.json('a2_m1.json')

In [16]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2020-02-17 20:06:20--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2020-02-17 20:06:20 (88.4 MB/s) - ‘rklib.py’ saved [2540/2540]



In [0]:
import zipfile

def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('a2_m1.json.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('a2_m1.json', zipf)
zipf.close()

In [0]:
!base64 a2_m1.json.zip > a2_m1.json.zip.base64

In [19]:
from rklib import submit
key = "1injH2F0EeiLlRJ3eJKoXA"
part = "wNLDt"
email = "sisokels@gmail.com"
token = "Wb8YujFos5bEGfAl" #(have a look here if you need more information on how to obtain the token https://youtu.be/GcDo0Rwe06U?t=276)

with open('a2_m1.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, token, key, part, [part], data)

Something went wrong, please have a look at the reponse of the grader
-------------------------
{"errorCode":"invalidEmailOrToken","message":"Invalid email or token.","details":{"learnerMessage":"You used an invalid email or your token may have expired. Please make sure you have entered all fields correctly. Try generating a new token if the issue still persists."}}
-------------------------
